# Import lib


In [1]:
import pandas as pd
import numpy as np
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input,Dense,Flatten,Conv2D,BatchNormalization,MaxPooling2D,Dropout, Add
from tensorflow.keras.models import Model

# useful for getting number of output classes
from glob import glob

import matplotlib.pyplot as plt
from matplotlib import image
from PIL import Image

# Dataset

In [2]:
def load_images(image_directory):
    images = []
    for filename in os.listdir(image_directory):
        if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
            image_path = os.path.join(image_directory, filename)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            image = cv2.resize(image, (224, 224))  # Resize the image to a desired size
            images.append(image)
    images = np.array(images)
    return images

# def load_images(image_directory):
#     images = []
#     for filename in os.listdir(image_directory):
#         if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
#             image_path = os.path.join(image_directory, filename)
#             image = cv2.imread(image_path)
#             image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
#             image = cv2.resize(image, (224, 224))  # Resize the image to a desired size

#             # Apply Frangi filter
#             image = frangi(image)

#             images.append(image)
#     images = np.array(images)
#     return images

In [3]:
# Define the image and label directories
train_image_directory = "New/Disease_Grading/Training_Set"
test_image_directory = "New/Disease_Grading/Testing_Set"
train_labels_csv_file = "New/Disease_Grading/Training_Labels.csv"
test_labels_csv_file = "New/Disease_Grading/Testing_Labels.csv"

In [4]:
import os
from skimage.filters import frangi

train_images = load_images(train_image_directory)
test_images = load_images(test_image_directory)

In [5]:
train_labels_df = pd.read_csv(train_labels_csv_file)
train_labels = train_labels_df['label'].values

test_labels_df = pd.read_csv(test_labels_csv_file)
test_labels = test_labels_df['label'].values

In [6]:
# Apply data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,      # Rotate images randomly by up to 20 degrees
    width_shift_range=0.1,  # Shift images horizontally by up to 10% of the image width
    height_shift_range=0.1, # Shift images vertically by up to 10% of the image height
    zoom_range=0.2,         # Zoom in or out on images by up to 20%
    horizontal_flip=True,   # Flip images horizontally
    vertical_flip=True     # Flip images vertically
)

datagen.fit(train_images)

In [7]:
num_samples = len(train_images)  # Replace 'train_images' with your actual training data
num_augmentations = 1 + (15 * 2) + (0.1 * 2 + 1) * (0.1 * 2 + 1) * 0.2 * 2
total_images = int(num_samples * num_augmentations)

print("Total images after data augmentation:", total_images)

Total images after data augmentation: 13040


In [8]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(train_labels)
num_classes = len(label_encoder.classes_)
categorical_labels = to_categorical(encoded_labels, num_classes)

# Model 

In [7]:
# # Define the model parameters using functional api
# # original model

# inputs = Input(shape=(224,224,3))
# cnn1 = Conv2D(filters=128, kernel_size=(11,11), strides=(4,4), activation='relu')(inputs)
# bnor1 = BatchNormalization()(cnn1)
# max1 = MaxPooling2D(pool_size=(2,2))(bnor1)

# cnn2 = Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same")(max1)
# bnor2 = BatchNormalization()(cnn2)
# max2 = MaxPooling2D(pool_size=(3,3))(bnor2)


# cnn3 = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same")(max2)
# bnor3 = BatchNormalization()(cnn3)

# cnn4 = Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(bnor3)
# bnor4 = BatchNormalization()(cnn4)

# cnn5 = Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(bnor4)
# add = Add()([cnn5,cnn3])
# bnor5 = BatchNormalization()(add)

# cnn6 = Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(bnor5)
# bnor6 = BatchNormalization()(cnn6)

# cnn7 = Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(bnor6)
# add1 = Add()([cnn7,bnor5,cnn3])
# bnor7 = BatchNormalization()(add1)
# max3 = MaxPooling2D(pool_size=(2,2))(bnor7)

# flat = Flatten()(max3)
# dense1  = Dense(1024, activation = 'relu')(flat)
# drop1 = Dropout(0.5)(dense1)
# dense2 = Dense(1024, activation = 'relu')(drop1)
# add2 = Add()([dense2,dense1])
# dense3 = Dense(1024, activation = 'relu')(add2)
# add3 = Add()([dense3,dense1])
# drop2 = Dropout(0.5)(add3)
# outputs = Dense(5, activation = 'softmax')(drop2)

# model = Model(inputs=inputs, outputs=outputs, name='UGP_model') # defining the model

In [8]:
# # resnet50
# # create empty model
# from tensorflow.keras.models import Sequential
# model = Sequential()

# # import resnet50
# pretrained_model= tf.keras.applications.ResNet50(include_top=False,
#                    input_shape=(224,224,3),
#                    pooling='max',classes=5,
#                    weights='imagenet')

# for layer in pretrained_model.layers:
#         layer.trainable=False

# # add layers
# model.add(pretrained_model)
# model.add(Flatten())
# model.add(Dense(521, activation='relu'))
# model.add(Dense(5, activation='softmax'))

In [7]:
# # densenet50
# from tensorflow.keras.models import Sequential

# model = Sequential()

# pretrained_model = tf.keras.applications.DenseNet121(
#     include_top=False,
#     weights="imagenet",
#     input_shape=(224,224,3),
#     pooling=None
# )

# for layers in pretrained_model.layers:
#     layers.trainable = False

# model.add(pretrained_model)
# model.add(Flatten())
# model.add(Dense(256, activation='relu'))
# model.add(Dense(5, activation='softmax'))

In [9]:
# vgg19
from tensorflow.keras.applications.vgg19 import VGG19

IMAGE_SHAPE = [224,224]

# we need to keep the weights same as the original model and just drop first and last layer
# Here we will be using imagenet weights
vgg = VGG19(input_shape=IMAGE_SHAPE + [3], weights='imagenet', include_top=False)

for layer in vgg.layers:
    layer.trainable = False

# changing the last layers since we need 5 classes rather than 1000 which it already does
x = Flatten()(vgg.output)
x = Dense(521, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout with 50% rate
prediction = Dense(num_classes, activation='softmax')(x)
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [10]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = Adam(learning_rate=0.001),
    metrics = ['accuracy']
)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [11]:
batch_size = 64
epochs = 50

model.fit(train_images, categorical_labels, batch_size=batch_size, epochs=epochs)

Epoch 1/50
7/7 [==============================] - 32s 2s/step - loss: 107.6376 - accuracy: 0.3002
Epoch 2/50
7/7 [==============================] - 4s 625ms/step - loss: 42.3157 - accuracy: 0.3487
Epoch 3/50
7/7 [==============================] - 4s 626ms/step - loss: 21.5368 - accuracy: 0.4528
Epoch 4/50
7/7 [==============================] - 4s 627ms/step - loss: 12.1450 - accuracy: 0.5521
Epoch 5/50
7/7 [==============================] - 4s 628ms/step - loss: 3.2680 - accuracy: 0.6707
Epoch 6/50
7/7 [==============================] - 4s 629ms/step - loss: 1.1498 - accuracy: 0.7119
Epoch 7/50
7/7 [==============================] - 4s 627ms/step - loss: 0.7802 - accuracy: 0.7240
Epoch 8/50
7/7 [==============================] - 4s 632ms/step - loss: 0.7046 - accuracy: 0.7337
Epoch 9/50
7/7 [==============================] - 4s 632ms/step - loss: 0.6512 - accuracy: 0.7797
Epoch 10/50
7/7 [==============================] - 4s 634ms/step - loss: 0.5633 - accuracy: 0.7554
Epoch 11/50
7/7 

In [12]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(test_labels)
num_classes = len(label_encoder.classes_)
test_categorical_labels = to_categorical(encoded_labels, num_classes)

In [13]:
# Vgg19 augemented 

test_loss, test_accuracy = model.evaluate(test_images, test_categorical_labels)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

4/4 [==============================] - 14s 1s/step - loss: 2.9503 - accuracy: 0.4757
Test Loss: 2.9503
Test Accuracy: 0.4757


In [13]:
#original

test_loss, test_accuracy = model.evaluate(test_images, test_categorical_labels)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

4/4 [==============================] - 1s 90ms/step - loss: 2.0589 - accuracy: 0.4854
Test Loss: 2.0589
Test Accuracy: 0.4854


In [13]:
#rsnet

test_loss, test_accuracy = model.evaluate(test_images, test_categorical_labels)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

4/4 [==============================] - 3s 295ms/step - loss: 1.8863 - accuracy: 0.4951
Test Loss: 1.8863
Test Accuracy: 0.4951


In [11]:
#densenet

test_loss, test_accuracy = model.evaluate(test_images, test_categorical_labels)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

4/4 [==============================] - 6s 688ms/step - loss: 8.0364 - accuracy: 0.4272
Test Loss: 8.0364
Test Accuracy: 0.4272


In [13]:
#vgg

test_loss, test_accuracy = model.evaluate(test_images, test_categorical_labels)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

4/4 [==============================] - 11s 1s/step - loss: 5.1439 - accuracy: 0.4757
Test Loss: 5.1439
Test Accuracy: 0.4757
